<a href="https://colab.research.google.com/github/scmassey/model-sensitivity-analysis/blob/master/LHS_PRCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Latin Hypercube Sampling & Partial Rank Correlation Coefficients  <br/> *~ a method for analyzing model sensitivity to parameters ~*

#### Importing packages that will be used.

In [0]:
import numpy as np

from scipy import special

import random

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

import pandas as pd

import matplotlib.pyplot as plt


#### Specify the number of parameters to sample and the number of samples to draw from each parameter distribution.

In [0]:
#  Number of parameters to sample (don't include add'l parameters you would like to leave fixed): 
parameterCount = 2;

#  Number of samples to draw (recommend 25 to 1000; Higher numbers yield better results, but also takes longer):
sampleCount = 100; 

#### Define the function for drawing samples from a user-specified parameter distribution.

In [0]:
def sampleDistrib(modelParamName,distrib,distribSpecs): 
    
    if distrib == 'uniform':
        
#         mymin = widgets.FloatText(
#                 value=0,
#                 description='Minimum:',
#                 disabled=False
#               )
#         mymax = widgets.FloatText(
#                 value=5,
#                 description='Maximum:',
#                 disabled=False
#               )
        
#         display(mymin, mymax)
        
#         mmin = mymin.value
#         mmax = mymax.value

        mmin = distribSpecs[0]
        mmax = distribSpecs[1]
        
        intervalwidth = (mmax - mmin) / sampleCount        # width of sampling interval
        
        samples = []
        
        for sample in range(sampleCount):
            
            lower = mmin + intervalwidth * (sample-1)      # lower bound of sampling interval
            upper = mmin + intervalwidth * (sample)        # upper bound of sampling interval
            
            sampleVal = np.random.uniform(lower, upper)       # draw a random sample within the interval
            
            samples.append(sampleVal)

    
    
    elif distrib == 'normal':
        
#         mymean = widgets.FloatText(
#                 value=1,
#                 description='Mean:',
#                 disabled=False
#               ) 
#         myvar  = widgets.FloatText(
#                 value=1,
#                 description='Variance:',
#                 disabled=False
#               ) 
        
#         display(mymean, myvar)
        
#         mmean = mymean.value
#         mvar = myvar.value
        
        mmean= distribSpecs[0]
        mvar = distribSpecs[1]
        
        lower = mvar * np.sqrt(2) * special.erfinv(-0.9999) + mmean   # lowest bound of parameter values, 
                                                           # initial sampling inverval lower bound
        samples = []
        
        for sample in range(sampleCount):
            
            upper = mvar * np.sqrt(2) * special.erfinv(2 / sampleCount + special.erf(lower - mmean) / (mvar * np.sqrt(2))) + mmean   # upper bound of sampling interval
 
            sampleVal = np.random.uniform(lower, upper)       # draw a random sample within the interval
            
            samples.append(sampleVal)

            lower = upper         # set current upper bound as the lower bound for the next iteration
            

    
    elif distrib == 'triangle':
        
#         mymin  = widgets.FloatText(
#                 value=0,
#                 description='Minimum:',
#                 disabled=False
#               )
#         mymax  = widgets.FloatText(
#                 value=2,
#                 description='Maximum:',
#                 disabled=False
#               )
#         mymode = widgets.FloatText(
#                 value=1,
#                 description='Mode:',
#                 disabled=False
#               )
        
#         display(mymin, mymax, mymode)
        
#         mmin = mymin.value
#         mmax = mymax.value
#         mmode=mymode.value
        
        mmin = distribSpecs[0]
        mmax = distribSpecs[1]
        mmode= distribSpecs[2]
    
        samples = []
        
        for sample in range(sampleCount):
            
            intervalarea = 1/sampleCount 
            
            ylower = intervalarea*(sample-1)  # looking at cdf read off area as y values, and convert
            yupper = intervalarea*(sample)    # to get x values, which are same in cdf as in pdf
        
        
            # Check to see if y values = cdf(x <= mmode) for calculating correxponding x values:
            
            if ylower <= ((mmode - mmin)/(mmax - mmin)):     
                lower = np.sqrt(ylower * (mmax - mmin) * (mmode - mmin)) + mmin 

            else:
                lower = mmax - np.sqrt((1 - ylower) * (mmax - mmin) * (mmax - mmode))

            
            if yupper <= ((mmode - mmin)/(mmax - mmin)):    
                upper = np.sqrt(yupper * (mmax - mmin) * (mmode - mmin)) + mmin; 

            else:
                upper = mmax - np.sqrt((1 - yupper) * (mmax - mmin) * (mmax - mmode));  

                
            sampleVal = np.random.uniform(lower, upper)   
            
            samples.append(sampleVal)
            
            
#     parameters[modelParamName] = samples
    
    plt.hist(samples,density=1, bins=5) 
    plt.show()
    
    # TO DO: This is where I would also add some plot commands to show the distribution specified, the intervals
    # and where the samples were drawn (latter for low sampleCount only) - or maybe just show a histogram?
    
    return samples # TO DO: Or do I want to return parameters here?

#### Call to the above function using interactive - user can use the boxes and dropdowns to specify parameter distributions and draw samples. 
## TRY USING ASYNCIO TO PAUSE FOR USER INPUT BEFORE GENERATING THE SAMPLES

In [0]:
parameters = {}
for param in range(parameterCount):
    myDict={}
    
    s=str(param)
    modelParamName = widgets.Text(value='Type parameter ' + s + ' name here',description='Name:')
    distrib = widgets.Dropdown(options = ['uniform','normal','triangle'],description='Distribution:',)

    display(modelParamName, distrib)
    
    distribSpecs = [0,5]

    if distrib.value == 'uniform':

            mymin = widgets.FloatText(
                    value=0,
                    description='Minimum:',
                    disabled=False
                  )
            mymax = widgets.FloatText(
                    value=5,
                    description='Maximum:',
                    disabled=False
                  )

            display(mymin, mymax)

            mmin = mymin.value
            mmax = mymax.value

            distribSpecs = [mmin,mmax]
            
            myDict[modelParamName.value] = sampleDistrib(modelParamName.value,distrib.value,distribSpecs)

    elif distrib.value == 'normal':

            mymean = widgets.FloatText(
                    value=1,
                    description='Mean:',
                    disabled=False
                  ) 
            myvar  = widgets.FloatText(
                    value=1,
                    description='Variance:',
                    disabled=False
                  ) 

            display(mymean, myvar)

            mmean = mymean.value
            mvar = myvar.value

            distribSpecs = [mmean,mvar]
            
            myDict[modelParamName.value] = sampleDistrib(modelParamName.value,distrib.value,distribSpecs)

    elif distrib.value == 'triangle':

            mymin  = widgets.FloatText(
                    value=0,
                    description='Minimum:',
                    disabled=False
                  )
            mymax  = widgets.FloatText(
                    value=2,
                    description='Maximum:',
                    disabled=False
                  )
            mymode = widgets.FloatText(
                    value=1,
                    description='Mode:',
                    disabled=False
                  )

            display(mymin, mymax, mymode)

            mmin = mymin.value
            mmax = mymax.value
            mmode=mymode.value

            distribSpecs = [mmin,mmax,mmode]
            
            myDict[modelParamName.value] = sampleDistrib(modelParamName.value,distrib.value,distribSpecs)

    parameters.update(myDict);
    

interactive(children=(Text(value='Type your parameter name here', description='modelParamName'), Dropdown(desc…

#### Randomly permute each set of parameter samples in order to randomly pair the samples to more fully sample the parameter space for the Monte Carlo simulations.

In [0]:
#  Now need to put samples in a matrix and randomly permute the columns... after I figure out how to go
#  through a loop for all the different parameters (probably in the function call above?) I just don't understand
#  what is being passed from that in order to put them in the matrix...

LHSparams=[]

for p in parameters:
    
    temp = parameters[p]
    random.shuffle(temp)
    
    LHSparams.append(temp)

AttributeError: 'interactive' object has no attribute 'size'

## Define your model function here -
### A trivial example has been provided,  but in practice may call an ode or pde solver.

In [0]:
def testlinear(x,sampledParams,unsampledParams):

    m = sampledParams[0]
    b = sampledParams[1]
    
    a = unsampledParams[0]

    y = m * x + b + a;

    return y    

#### Run Monte Carlo simulations  
## TO DO: I need to modify matlab code dropped in below; don't forget to add code to plot outputs with error bars

In [0]:
# % Specify the independent variables you will pass to the model function in a vector
# x = linspace(0,10,101); % spatial domain for my testlinear function
# % t = linspace(0,17,171);% time vector w/ number of time steps desired for kineticpdgfpdes

# % Specify values of any unsampled parameters to pass to the function
# unsampledps =  2; 

# % Don't forget any solver parameters if needed: 
# % w = 2; %spherical symmetry = 2 for pdepe.m to solve kineticpdgfpdes

# Simdata = struct; %place where all output data will be stored for computing PRCCs 

# tic % start measuring time to solve equations
# for j=1:N   
#     sampledparams=A(j,1:M);
#     fprintf('Parameters passed for sample: ');fprintf('%u',j);fprintf(' of ');fprintf('%u\n',N);
    
# %  EDIT THE FOLLOWING FUNCTION CALL--can name multiple outputs as in commented example below
# %  Also make sure everything passes to your function correctly and all initial conditions,
# %  etc. are specified before entering this for loop over j.
#     Simdata(j).y=testlinear(x,sampledparams,unsampledps);
# % %     sol = pdepe(w,@kineticpdgfpdes,@pdgfpdesic,@pdgfpdesbc,x,t); %make sure the right set of pdes are used
# % %     Simdata(...



# TL=length(x);
# %Creating L, matrix of all outputs at each time, for each of the N samples
# L=zeros(TL,N);
# R=zeros(TL,N);  
# for i=1:TL
#     for k=1:N
#             L(i,k)=Simdata(k).y(i);
#     end
# end
# Z=L';% {want mean and std dev computed with each day kept together, looking at variability across samples,
#      % {and returned as a row vector rather than a column vector (why use Z=L' instead of L)
# pause(.2)

# figure(6)       
# Y=mean(Z);   
# E=std(Z);
# errorbar(x,Y,E)
# xlabel('x')
# ylabel(labelstring)
# title(['Error bar plot of ',labelstring,' from LHS simulations']);
# pause(5)
# figurelabel1=(['LHSgeneral-N',num2str(N),'-',labelstring,'-ErrorbarPlot.fig']);
# figurelabel2=(['LHSgeneral-N',num2str(N),'-',labelstring,'-ErrorbarPlot.png']);
# saveas(gcf,figurelabel1);
# saveas(gcf,figurelabel2);

#### Compute partial rank correlation coefficients to compare simulation outputs with parameters
## TO DO: Next need PRCC codes... then plots of those.

#### BELOW CONTAINS EXAMPLES AND SUCH TO KEEP HANDY IN CASE NEED TO BORROW :) 

In [0]:
# Initializing matrices that will hold samples for the Monte Carlo simulations later
A=np.zeros(sampleCount,parameterCount)
B=np.zeros(sampleCount,parameterCount)   #,sampleCount):

#  TO DO: B(n,m)=parameters(m).sampleVal(n); %store the sample value in the B matrix

# To add with user input tools (widgets and the like) - will have to initialize first, 
# then update the "append" as each is named and sampled - or might need to specify field
# for each if user is actively changing (?)
parameters = []
# This is where user interface things will pop up... 
parameters.append({'name': 'slope', 'samples': [1, 2, 3, 4]})
parameters.append({'name': 'intercept', 'samples':[2, 5, 7, 11]})

myMap = {}

test = "foo"

myMap[test] = "bar"

print(myMap)


sampleCount = 10

samples = []

for sample in range(sampleCount):
    myDict = {}
    myDict.D = getRandomStuff()
    myDict.rho = getOtherRandomStff();
    
    samples.append(myDict);

In [0]:
mmin = widgets.FloatText(
        value=0,
        description='Minimum:',
        disabled=False
       )
mmax = widgets.FloatText(
        value=0,
        description='Maximum:',
        disabled=False
)
mmode = widgets.FloatText(
        value=0,
        description='Mode:',
        disabled=False
)

display(mmin, mmax, mmode)

FloatText(value=0.0, description='Minimum:')

FloatText(value=0.0, description='Maximum:')

FloatText(value=0.0, description='Mode:')

NameError: name 'lower' is not defined